In [2]:
%pip install --quiet --upgrade langchain langchain-openai langchain-pinecone pinecone-client pypdf transformers python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install boto3 --quiet

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install amazon-textract-caller

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install amazon-textract-textractor

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install -qU langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install python-dotenv

In [10]:
pip install -qU langchain-huggingface

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install -qU pypdf

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install -qU langchain-unstructured

Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install --upgrade --quiet langchain-unstructured unstructured-client unstructured "unstructured[pdf]" python-magic

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install "langchain-unstructured[local]"

  Using cached unstructured-0.15.14-py3-none-any.whl.metadata (29 kB)
  Using cached unstructured_inference-0.7.36-py3-none-any.whl.metadata (5.9 kB)
Using cached unstructured-0.15.14-py3-none-any.whl (2.1 MB)
Using cached unstructured_inference-0.7.36-py3-none-any.whl (56 kB)
  Attempting uninstall: unstructured
    Found existing installation: unstructured 0.16.11
    Uninstalling unstructured-0.16.11:
      Successfully uninstalled unstructured-0.16.11
  Attempting uninstall: unstructured-inference
    Found existing installation: unstructured-inference 0.8.1
    Uninstalling unstructured-inference-0.8.1:
      Successfully uninstalled unstructured-inference-0.8.1
Note: you may need to restart the kernel to use updated packages.


In [15]:
import getpass
import os
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_unstructured import UnstructuredLoader

In [ ]:
load_dotenv(find_dotenv(), override=True)
# input keys below
OPENAI_API_KEY=""
PINECONE_API_KEY=""
LANGCHAIN_API_KEY=""
UNSTRUCTURED_API_KEY=""

if "UNSTRUCTURED_API_KEY" not in os.environ:
    os.environ["UNSTRUCTURED_API_KEY"] = getpass.getpass("Unstructured API Key:")

llm = ChatOpenAI(model="gpt-4o-mini")


In [ ]:
# AWS Credentials and S3 Configuration
# input keys below
AWS_ACCESS_KEY=""
AWS_SECRET_KEY=""
AWS_REGION = "us-east-2"  # Change to your preferred region
BUCKET_NAME = "my-textbook-bucket"  # Replace with your S3 bucket name
FILE_NAME = "textbook.pdf"  # Local file
S3_KEY = "uploads/textbook.pdf"  # Path in S3 bucket

In [ ]:
print("LANGCHAIN_API_KEY", LANGCHAIN_API_KEY)
print("OPENAI_API_KEY", OPENAI_API_KEY)
print("UNSTRUCTURED_API_KEY", UNSTRUCTURED_API_KEY)
print("PINECONE_API_KEY", PINECONE_API_KEY)
print("AWS_ACCESS_KEY", AWS_ACCESS_KEY)
print("AWS_SECRET_KEY", AWS_SECRET_KEY)

In [65]:
import boto3

# Upload PDF to S3
s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)
s3.upload_file(FILE_NAME, BUCKET_NAME, S3_KEY)
print(f"Uploaded {FILE_NAME} to {BUCKET_NAME}/{S3_KEY}")

Uploaded textbook.pdf to my-textbook-bucket/uploads/textbook.pdf


In [72]:

# Start Textract job
textract = boto3.client('textract', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY, region_name=AWS_REGION)
response = textract.start_document_text_detection(
    DocumentLocation={'S3Object': {'Bucket': BUCKET_NAME, 'Name': S3_KEY}}
)
job_id = response['JobId']
print(f"Started Textract job with ID: {job_id}")

Started Textract job with ID: a3024216dfb3289b68534b1add08c36f986c7040eaf08c1537eeecb06498d547


In [74]:
import time
# Wait and retrieve job results
def get_textract_results(job_id):
    while True:
        response = textract.get_document_text_detection(JobId=job_id)
        status = response['JobStatus']
        if status in ['SUCCEEDED', 'FAILED']:
            break
        print("Waiting for Textract to complete...")
        time.sleep(5)
    return response

result = get_textract_results(job_id)
pages = []
for item in result['Blocks']:
    if item['BlockType'] == 'LINE':
        pages.append(item['Text'])
print("Extracted text from the PDF.")

Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Waiting for Textract to complete...
Extracted text from the PDF.


In [75]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create document loader and split into chunks
text_content = "\n".join(pages)
text_loader = TextLoader(text_content)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_text(text_content)
print(f"Loaded and split {len(docs)} document chunks.")

Loaded and split 14 document chunks.


In [76]:
# Initialize components
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
pinecone = Pinecone(api_key=PINECONE_API_KEY)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [105]:
import os
from pinecone import Pinecone, Index, ServerlessSpec

# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define index name and embedding dimension
index_name = "textbook-index"
embedding_dimension = 768

# Check if the index already exists
if index_name not in pc.list_indexes().names():
    print(f"Creating index: {index_name}")
    pc.create_index(
        name=index_name,
        dimension=embedding_dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",  # Cloud provider
            region="us-east-1"  # Supported region for your plan
        )
    )
    print(f"Created index: {index_name}")

# Describe the index and retrieve the host
try:
    index_description = pc.describe_index(index_name)
    print("Index description:", index_description)

    # Retrieve host directly from 'host' field
    host = index_description["host"]
    if not host:
        raise ValueError("Host endpoint is empty. Ensure the index creation was successful.")
    print("Retrieved host:", host)

except Exception as e:
    raise ValueError(f"Error retrieving host for index {index_name}: {e}")

# Connect to the index using the Index class
index = Index(api_key=PINECONE_API_KEY, host=host)
print(f"Connected to index: {index_name}")


Index description: {'deletion_protection': 'disabled',
 'dimension': 768,
 'host': 'textbook-index-t7cj4ax.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'textbook-index',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}
Retrieved host: textbook-index-t7cj4ax.svc.aped-4627-b74a.pinecone.io
Connected to index: textbook-index


In [20]:
from langchain_community.document_loaders import AmazonTextractPDFLoader
import boto3

# Configure AWS Textract client
textract_client = boto3.client(
    'textract',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

# Define S3 file path
s3_path = f"s3://{BUCKET_NAME}/{S3_KEY}"

# Initialize Textract Loader
loader = AmazonTextractPDFLoader(
    file_path=s3_path,
    client=textract_client  # Pass the configured boto3 client
)

# Load and process the document
docs = loader.load()
print(f"Loaded {len(docs)} pages of text from Amazon Textract.")

# Example output of the first page
print(docs[0].page_content)


Loaded 188 pages of text from Amazon Textract.
N.R. Galloway


and W.M.K. Amoaku


Common Eye Diseases


and their Management


Second Edition


Springer



In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

# Text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)

# Initialize Pinecone
pinecone = Pinecone(api_key=PINECONE_API_KEY)
index_name = "textbook-index"
if index_name not in pinecone.list_indexes().names():
    pinecone.create_index(index_name, dimension=768)
index = pinecone.Index(index_name)
vector_store = PineconeVectorStore(embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"), index=index)

# Add chunks to Pinecone
vector_store.add_documents(chunks)
print("Textbook successfully indexed.")

/var/folders/6h/16gn0szx5dv9mhd7v9m0w5jh0000gn/T/ipykernel_5731/1044355300.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  vector_store = PineconeVectorStore(embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"), index=index)


Textbook successfully indexed.


In [24]:
!pip install langgraph


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [48]:
import base64

import httpx
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/07/Acute_angle_closure_glaucoma.JPG/440px-Acute_angle_closure_glaucoma.JPG"
model = ChatOpenAI(model="gpt-4o")
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
message = HumanMessage(
    content=[
        {"type": "text", "text": "Here is an image of an eyeball. There are 6 diseases it could be: a - f. Describe this photo the way an eye doctor would do it:\n"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = model.invoke([message])
print(response.content)

The image shows both eyes with several notable features:

1. **Conjunctival Hyperemia**: There is significant redness in the conjunctiva, indicating increased blood flow, often associated with inflammation or irritation.

2. **Proptosis**: The eyes appear to be protruding slightly, which could be indicative of orbital involvement or increased orbital pressure.

3. **Eyelid Retraction**: The upper eyelids seem retracted, exposing more of the sclera above the cornea.

4. **Periorbital Swelling**: There may be some swelling around the eyes, which could suggest inflammation or fluid retention.

These observations could be consistent with conditions such as Graves' disease, conjunctivitis, or other orbital/ocular pathologies. Further examination, including a detailed history and possibly imaging or blood tests, would be necessary for accurate diagnosis.


In [49]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
def get_disease_probabilities() -> str:
    nn_output = {
        "a Diabetic retinopathy": 0.1,
        "b glaucoma": 0.05,
        "c cataracts": 0.7,
        "d age-related macular degeneration": 0.05,
        "e pathological myopia": 0.05,
        "f hypertensive retinopathy": 0.05
    }
    
    probs_str = "\n".join([f"{d}: {p}" for d, p in nn_output.items()])
    print("Disease probabilities generated successfully.")
    return probs_str

probabilities_str = get_disease_probabilities()

final_prompt = (
        "You are a doctor nerd. "
        "We have a state-of-the-art machine learning model, and this is what it says the probabilities "
        "of each disease are:\n"
        f"{probabilities_str}\n\n"
        "Given all your years of being a doctor, what do you think? "
        "Format: Keep it one word / from 1-6 or a-f."
    )




Disease probabilities generated successfully.


In [52]:
for step in graph.stream(
    {"question": final_prompt}, stream_mode="updates"
):
    print(f"{step}\n\n----------------\n")

{'retrieve': {'context': [Document(id='8f903a4e-2b98-4f3a-99b8-a873dff39327', metadata={'page': 182.0, 'source': 's3://my-textbook-bucket/uploads/textbook.pdf'}, page_content='109\n\n\ntime spent in hospital\n\n\n78\n\n\ndiabetes 147-8\n\n\ntoxic 74\n\n\ndiseases of\n\n\n41-6\n\n\ntraumatic 73, 77\n\n\nmelanoma\n\n\n111\n\n\nwhen to operate 76-7\n\n\nConjunctivitis 41, 55-6\n\n\nCavernous haemangioma\n\n\n110,\n\n\n112\n\n\nacute haemorrhagic\n\n\n44\n\n\nCentral retinal vein thrombosis\n\n\n89\n\n\nadenoviral 44\n\n\nCentral retinal vein occlusion\n\n\n89\n\n\nallergic 44-6\n\n\nCerebrospinal fluid\n\n\n63\n\n\nangular 43\n\n\nChalazion 108\n\n\nbacterial 42-3\n\n\nChemical injuries 119\n\n\ncauses 42\n\n\nChemosis 153\n\n\nchlamydia 43\n\n\nChildren 141-6'), Document(id='36cd8f33-dd1b-4c5d-be01-d4da7d817049', metadata={'page': 182.0, 'source': 's3://my-textbook-bucket/uploads/textbook.pdf'}, page_content='109\n\n\ntime spent in hospital\n\n\n78\n\n\ndiabetes 147-8\n\n\ntoxic 74\n\n\n